In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import plotly.graph_objs as go
%matplotlib inline

# Imports in order to be able to use Plotly offline.
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


print(__version__)
init_notebook_mode(connected=True)

# Import the Sample worksheet with acquisition dates and initial cost basis:
portfolio_df = pd.read_excel('Sample stocks acquisition dates_costs.xlsx', sheet_name='Sample')
#mac: '/Users/SlavOK/Google Drive/Python/Sample stocks acquisition dates_costs.xlsx'
#win: 'C:\\Users\\spiris\\Google Drive\\Python\\Sample stocks acquisition dates_costs.xlsx'

portfolio_df.head(10)



3.2.1


,Acquisition Date,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year
0,2013-02-07,AAPL,125,65.40,8175.00,2017-12-29
1,2014-02-27,JNJ,100,81.90,8190.00,2017-12-29
2,2013-06-04,MCD,100,84.99,8499.00,2017-12-29
3,2015-12-14,MTCH,600,13.63,8178.00,2017-12-29
4,2016-01-14,NFLX,75,108.71,8153.25,2017-12-29
5,2013-08-14,WMT,125,68.30,8537.50,2017-12-29
6,2013-12-13,FB,150,53.32,7998.00,2017-12-29
7,2015-01-05,TWTR,225,36.38,8185.50,2017-12-29


In [2]:
# Date Ranges for SP 500 and for all tickers
# Modify these date ranges each week.

# The below will pull back stock prices from the start date until end date specified.

start_sp = datetime.datetime(2013, 1, 1)
end_sp = datetime.datetime(2018, 3, 9)

# This variable is used for YTD performance.
end_of_last_year = datetime.datetime(2017, 12, 29)

# These are separate if for some reason want different date range than SP.
stocks_start = datetime.datetime(2013, 1, 1)
stocks_end = datetime.datetime(2018, 3, 9)

In [3]:
from pandas_datareader import data as pdr

import pandas as pd
import pandas_datareader.data as web

sp500 = web.DataReader('^GSPC', 'yahoo', start_sp, end_sp)
sp500.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2013-01-02,1462.430054,1426.189941,1426.189941,1462.420044,4202600000,1462.420044
2013-01-03,1465.469971,1455.530029,1462.420044,1459.369995,3829730000,1459.369995
2013-01-04,1467.939941,1458.989990,1459.369995,1466.469971,3424290000,1466.469971
2013-01-07,1466.469971,1456.619995,1466.469971,1461.890015,3304970000,1461.890015
2013-01-08,1461.890015,1451.640015,1461.890015,1457.150024,3601600000,1457.150024


In [4]:
sp_500_adj_close = sp500[['Adj Close']].reset_index()
sp_500_adj_close_start = sp_500_adj_close[sp_500_adj_close['Date']==end_of_last_year]
sp_500_adj_close.tail()

,Date,Adj Close
1301,2018-03-05,2720.939941
1302,2018-03-06,2728.120117
1303,2018-03-07,2726.800049
1304,2018-03-08,2738.969971
1305,2018-03-09,2786.570068


In [5]:
tickers = portfolio_df['Ticker'].unique()

def get(tickers, startdate, enddate):
    def data(ticker):
        return ((web.DataReader(ticker, 'yahoo', start=startdate, end=enddate)))
    datas = map(data,tickers) # apply the function to objects
    return(pd.concat(datas, keys = tickers, names = ['Ticker', 'Date']))

all_data = get(tickers, stocks_start, stocks_end)

In [6]:
# Also only pulling the ticker, date and adj. close columns for our tickers.

adj_close = all_data[['Adj Close']].reset_index()
adj_close.head()

# Grabbing the ticker close from the end of last year
adj_close_start = adj_close[adj_close['Date'] == end_of_last_year]

# Grab the latest stock close price
adj_close_latest = adj_close[adj_close['Date'] == stocks_end]
adj_close_latest.set_index('Ticker', inplace = True)

# Set portfolio index prior to merging with the adj close latest.
portfolio_df.set_index('Ticker', inplace = True)

portfolio_df.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year
Ticker,,,,,
AAPL,2013-02-07,125,65.40,8175.00,2017-12-29
JNJ,2014-02-27,100,81.90,8190.00,2017-12-29
MCD,2013-06-04,100,84.99,8499.00,2017-12-29
MTCH,2015-12-14,600,13.63,8178.00,2017-12-29
NFLX,2016-01-14,75,108.71,8153.25,2017-12-29


In [7]:
merged_portfolio = pd.merge(portfolio_df, adj_close_latest, left_index=True, right_index=True)

# The below creates a new column which is the ticker return; takes the latest adjusted close for each position
# and divides that by the initial share cost.

merged_portfolio['ticker return'] = merged_portfolio['Adj Close'] / merged_portfolio['Unit Cost'] - 1

merged_portfolio.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date,Adj Close,ticker return
Ticker,,,,,,,,
AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,178.662048,1.731836
JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,131.935715,0.610937
MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,155.278137,0.827017
MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,44.790001,2.286134
NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846


In [8]:
merged_portfolio.reset_index(inplace = True)

# Here we are merging the new dataframe with the sp500 adjusted closes since the sp start price based on 
# each ticker's acquisition date and sp500 close date.

merged_portfolio_sp = pd.merge(merged_portfolio, sp_500_adj_close, left_on = 'Acquisition Date', right_on = 'Date')
merged_portfolio_sp.head(2)

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date_x,Adj Close_x,ticker return,Date_y,Adj Close_y
0,AAPL,2013-02-07,125,65.4,8175.0,2017-12-29,2018-03-09,178.662048,1.731836,2013-02-07,1509.390015
1,JNJ,2014-02-27,100,81.9,8190.0,2017-12-29,2018-03-09,131.935715,0.610937,2014-02-27,1854.290039


In [9]:
del merged_portfolio_sp['Date_y']
merged_portfolio_sp.rename(columns = {'Date_x': 'Latest Date', 'Adj Close_x': 'Ticker Adj Close', 'Adj Close_y': 'SP 500 Initial Close'}, inplace = True)
merged_portfolio_sp['Equiv SP Shares'] = merged_portfolio_sp['Cost Basis'] / merged_portfolio_sp['SP 500 Initial Close']
merged_portfolio_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,178.662048,1.731836,1509.390015,5.416095
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,131.935715,0.610937,1854.290039,4.416785
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,155.278137,0.827017,1631.380005,5.209700
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,44.790001,2.286134,2021.939941,4.044631
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,4.242419


In [10]:
# We are joining the developing dataframe with the sp500 closes again, 
# this time with the latest close for SP

merged_portfolio_sp_latest = pd.merge(merged_portfolio_sp, sp_500_adj_close, left_on = 'Latest Date', right_on = 'Date')
merged_portfolio_sp_latest.head()

# Once again need to delete the new Date column added as it's redundant to Latest Date.  
# Modify Adj Close from the sp dataframe to distinguish it by calling it the SP 500 Latest Close.

del merged_portfolio_sp_latest['Date']
merged_portfolio_sp_latest.rename(columns={'Adj Close': 'SP 500 Latest Close'}, inplace=True)
merged_portfolio_sp_latest.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,178.662048,1.731836,1509.390015,5.416095,2786.570068
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,131.935715,0.610937,1854.290039,4.416785,2786.570068
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,155.278137,0.827017,1631.380005,5.209700,2786.570068
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,44.790001,2.286134,2021.939941,4.044631,2786.570068
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,4.242419,2786.570068


In [11]:
# Percent return of SP from acquisition date of position through latest trading day.
merged_portfolio_sp_latest['SP Return'] = merged_portfolio_sp_latest['SP 500 Latest Close'] / merged_portfolio_sp_latest['SP 500 Initial Close'] - 1

# This is a new column which takes the tickers return and subtracts the sp 500 equivalent range return.
merged_portfolio_sp_latest['Abs. Return Compare'] = merged_portfolio_sp_latest['ticker return'] - merged_portfolio_sp_latest['SP Return']

# This is a new column where we calculate the ticker's share value by multiplying the original quantity by the latest close.
merged_portfolio_sp_latest['Ticker Share Value'] = merged_portfolio_sp_latest['Quantity'] * merged_portfolio_sp_latest['Ticker Adj Close']

# We calculate the equivalent SP 500 Value if we take the original SP shares * the latest SP 500 share price.
merged_portfolio_sp_latest['SP 500 Value'] = merged_portfolio_sp_latest['Equiv SP Shares'] * merged_portfolio_sp_latest['SP 500 Latest Close']

# This is a new column where we take the current market value for the shares and subtract the SP 500 value.
merged_portfolio_sp_latest['Abs Value Compare'] = merged_portfolio_sp_latest['Ticker Share Value'] - merged_portfolio_sp_latest['SP 500 Value']

# This column calculates profit / loss for stock position.
merged_portfolio_sp_latest['Stock Gain / (Loss)'] = merged_portfolio_sp_latest['Ticker Share Value'] - merged_portfolio_sp_latest['Cost Basis']

# This column calculates profit / loss for SP 500.
merged_portfolio_sp_latest['SP 500 Gain / (Loss)'] = merged_portfolio_sp_latest['SP 500 Value'] - merged_portfolio_sp_latest['Cost Basis']

merged_portfolio_sp_latest.head()


,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss)
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,178.662048,1.731836,1509.390015,5.416095,2786.570068,0.846156,0.885679,22332.756042,15092.328747,7240.427295,14157.756042,6917.328747
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,131.935715,0.610937,1854.290039,4.416785,2786.570068,0.502769,0.108167,13193.571472,12307.680233,885.891239,5003.571472,4117.680233
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,155.278137,0.827017,1631.380005,5.209700,2786.570068,0.708106,0.118910,15527.813721,14517.193382,1010.620338,7028.813721,6018.193382
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,44.790001,2.286134,2021.939941,4.044631,2786.570068,0.378167,1.907967,26874.000549,11270.646349,15603.354200,18696.000549,3092.646349
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,4.242419,2786.570068,0.449949,1.598897,24858.000183,11821.797243,13036.202940,16704.750183,3668.547243


In [12]:
# Merge the overall dataframe with the adj close start of year dataframe for YTD tracking of tickers.
merged_portfolio_sp_latest_YTD = pd.merge(merged_portfolio_sp_latest, adj_close_start, on = 'Ticker')

# Deleting date again as it's an unnecessary column.  Explaining that new column is the Ticker Start of Year Close.
del merged_portfolio_sp_latest_YTD['Date']
merged_portfolio_sp_latest_YTD.rename(columns={'Adj Close': 'Ticker Start Year Close'}, inplace=True)

# Join the SP 500 start of year with current dataframe for SP 500 ytd comparisons to tickers.
merged_portfolio_sp_latest_YTD_sp = pd.merge(merged_portfolio_sp_latest_YTD, sp_500_adj_close_start, left_on='Start of Year', right_on='Date')

# Deleting another unneeded Date column.
del merged_portfolio_sp_latest_YTD_sp['Date']

# Renaming so that it's clear this column is SP 500 start of year close.
merged_portfolio_sp_latest_YTD_sp.rename(columns={'Adj Close': 'SP Start Year Close'}, inplace=True)

# YTD return for portfolio position.
merged_portfolio_sp_latest_YTD_sp['Share YTD'] = merged_portfolio_sp_latest_YTD_sp['Ticker Adj Close'] / merged_portfolio_sp_latest_YTD_sp['Ticker Start Year Close'] - 1

# YTD return for SP to run compares.
merged_portfolio_sp_latest_YTD_sp['SP 500 YTD'] = merged_portfolio_sp_latest_YTD_sp['SP 500 Latest Close'] / merged_portfolio_sp_latest_YTD_sp['SP Start Year Close'] - 1


In [13]:
merged_portfolio_sp_latest_YTD_sp = merged_portfolio_sp_latest_YTD_sp.sort_values(by='Ticker', ascending=True)

# Cumulative sum of original investment
merged_portfolio_sp_latest_YTD_sp['Cum Invst'] = merged_portfolio_sp_latest_YTD_sp['Cost Basis'].cumsum()

# Cumulative sum of Ticker Share Value (latest FMV based on initial quantity purchased).
merged_portfolio_sp_latest_YTD_sp['Cum Ticker Returns'] = merged_portfolio_sp_latest_YTD_sp['Ticker Share Value'].cumsum()

# Cumulative sum of SP Share Value (latest FMV driven off of initial SP equiv purchase).
merged_portfolio_sp_latest_YTD_sp['Cum SP Returns'] = merged_portfolio_sp_latest_YTD_sp['SP 500 Value'].cumsum()

# Cumulative CoC multiple return for stock investments
merged_portfolio_sp_latest_YTD_sp['Cum Ticker ROI Mult'] = merged_portfolio_sp_latest_YTD_sp['Cum Ticker Returns'] / merged_portfolio_sp_latest_YTD_sp['Cum Invst'] - 1

merged_portfolio_sp_latest_YTD_sp.head(2)


,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult
0,AAPL,2013-02-07,125,65.40,8175.0,2017-12-29,2018-03-09,178.662048,1.731836,1509.390015,...,14157.756042,6917.328747,167.308640,2673.610107,0.067859,0.04225,8175.0,22332.756042,15092.328747,1.731836
6,FB,2013-12-13,150,53.32,7998.0,2017-12-29,2018-03-09,185.229996,2.473931,1775.319946,...,19786.499359,4555.786405,176.460007,2673.610107,0.049700,0.04225,16173.0,50117.255402,27646.115152,2.098822


In [14]:
# Need to factor in that some positions were purchased much more recently than others.
# Join adj_close dataframe with portfolio in order to have acquisition date.

portfolio_df.reset_index(inplace = True)

adj_close_acq_date = pd.merge(adj_close, portfolio_df, on = 'Ticker')

# delete_columns = ['Quantity', 'Unit Cost', 'Cost Basis', 'Start of Year']
del adj_close_acq_date['Quantity']
del adj_close_acq_date['Unit Cost']
del adj_close_acq_date['Cost Basis']
del adj_close_acq_date['Start of Year']

# Sort by these columns in this order in order to make it clearer where compare for each position should begin.
adj_close_acq_date.sort_values(by=['Ticker', 'Acquisition Date', 'Date'], ascending=[True, True, True], inplace=True)

# Anything less than 0 means that the stock close was prior to acquisition.
adj_close_acq_date['Date Delta'] = adj_close_acq_date['Date'] - adj_close_acq_date['Acquisition Date']
adj_close_acq_date['Date Delta'] = adj_close_acq_date[['Date Delta']].apply(pd.to_numeric)

# Modified the dataframe being evaluated to look at highest close which occurred after Acquisition Date (aka, not prior to purchase).
adj_close_acq_date_modified = adj_close_acq_date[adj_close_acq_date['Date Delta']>= 0]


In [15]:
# This pivot table will index on the Ticker and Acquisition Date, and find the max adjusted close.

adj_close_pivot = adj_close_acq_date_modified.pivot_table(index=['Ticker', 'Acquisition Date'], values = 'Adj Close', aggfunc=np.max)
adj_close_pivot.reset_index(inplace = True)

# Merge the adj close pivot table with the adj_close table in order to grab the date of the Adj Close High (good to know).

adj_close_pivot_merged= pd.merge(adj_close_pivot, adj_close, on=['Ticker', 'Adj Close'])
adj_close_pivot_merged.head()

,Ticker,Acquisition Date,Adj Close,Date
0,AAPL,2013-02-07,178.662048,2018-03-09
1,FB,2013-12-13,193.089996,2018-02-01
2,JNJ,2014-02-27,145.146500,2018-01-22
3,MCD,2013-06-04,175.027328,2018-01-26
4,MTCH,2015-12-14,44.790001,2018-03-09


In [16]:
# Merge the Adj Close pivot table with the master dataframe to have the closing high since you have owned the stock.
merged_portfolio_sp_latest_YTD_sp_closing_high = pd.merge(merged_portfolio_sp_latest_YTD_sp, adj_close_pivot_merged, on = ['Ticker', 'Acquisition Date'])

merged_portfolio_sp_latest_YTD_sp_closing_high.rename(columns={'Adj Close': 'Closing High Adj Close', 'Date': 'Closing High Adj Close Date'}, inplace=True)

merged_portfolio_sp_latest_YTD_sp_closing_high['Pct off High'] = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker Adj Close'] / merged_portfolio_sp_latest_YTD_sp_closing_high['Closing High Adj Close'] - 1 

merged_portfolio_sp_latest_YTD_sp_closing_high.head(3)

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,SP Start Year Close,Share YTD,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult,Closing High Adj Close,Closing High Adj Close Date,Pct off High
0,AAPL,2013-02-07,125,65.40,8175.0,2017-12-29,2018-03-09,178.662048,1.731836,1509.390015,...,2673.610107,0.067859,0.04225,8175.0,22332.756042,15092.328747,1.731836,178.662048,2018-03-09,0.000000
1,FB,2013-12-13,150,53.32,7998.0,2017-12-29,2018-03-09,185.229996,2.473931,1775.319946,...,2673.610107,0.049700,0.04225,16173.0,50117.255402,27646.115152,2.098822,193.089996,2018-02-01,-0.040706
2,JNJ,2014-02-27,100,81.90,8190.0,2017-12-29,2018-03-09,131.935715,0.610937,1854.290039,...,2673.610107,-0.036238,0.04225,24363.0,63310.826874,39953.795385,1.598647,145.146500,2018-01-22,-0.091017


In [17]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['ticker return'][0:10],
    name = 'Ticker Total Return')


trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['SP Return'][0:10],
    name = 'SP500 Total Return')

data = [trace1,trace2]
        
layout = go.Layout(title = 'Total Return vs S&P 500'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=0.8,y=1)
    )
fig = go.Figure(data=data, layout=layout)
        
iplot(fig)
        
        

In [18]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Stock Gain / (Loss)'][0:10],
    name = 'Ticker Total Return ($)')

trace2 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['SP 500 Gain / (Loss)'][0:10],
    name = 'SP 500 Total Return ($)')

trace3 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['ticker return'][0:10],
    name = 'Ticker Total Return %',
    yaxis='y2')

data = [trace1, trace2, trace3]

layout = go.Layout(title = 'Gain / (Loss) Total Return vs S&P 500'
    , barmode = 'group'
    , yaxis=dict(title='Gain / (Loss) ($)')
    , yaxis2=dict(title='Ticker Return', overlaying='y', side='right', tickformat="0.2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.75,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [19]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Invst'],
    # mode = 'lines+markers',
    name = 'Cum Invst')

trace2 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum SP Returns'],
    # mode = 'lines+markers',
    name = 'Cum SP500 Returns')

trace3 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Ticker Returns'],
    # mode = 'lines+markers',
    name = 'Cum Ticker Returns')

trace4 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Ticker ROI Mult'],
    # mode = 'lines+markers',
    name = 'Cum ROI Mult'
    , yaxis='y2')


data = [trace1, trace2, trace3, trace4]

layout = go.Layout(title = 'Total Cumulative Investments Over Time'
    , barmode = 'group'
    , yaxis=dict(title='Returns')
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.4,y=1)
    , yaxis2=dict(title='Cum ROI Mult', overlaying='y', side='right')               
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [20]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Pct off High'][0:10],
    name = 'Pct off High')

data = [trace1]
        
layout = go.Layout(title = 'Adjusted Close % off of High'
    , barmode = 'group'
    , yaxis=dict(title='% Below Adj Close High', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=0.8,y=1)
    )
fig = go.Figure(data=data, layout=layout)
        
iplot(fig)